In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Super_resolution/brainSR')

In [ ]:
from data.image_list import create_image_list
from data.dataset import create_dataset
from data.data_loader import create_dataloader
import torch
import time
import random
from collections import OrderedDict
import math
from math import log10
from utils import util, convert, metric

In [ ]:
opt = {
    "name": "espcn_x4_scheduler",
    "model": "espcn",
    "gpu_ids": [0],
    "upscale_factor":4,
    "device": "cuda",

    "datasets":
        {
            "path":
                {
                "root": "data/BrainTumour",
                "folders": ["imagesTr"]
                },
            "train":
                {
                "phase": "train",
                "batch_size": 64,
                "use_shuffle": True,
                "upscale_factor": 4,
                "scale": True,
                # "transform_H": {},
                "no_of_images": 450,
                "n_workers": 1
                },
            "valid":
                {
                "phase": "valid",
                "no_of_images": 34,
                "scale": True,
                "depth_padding": 1,
                "upscale_factor": 4                
                # "transform_H": {}
                }
    },

    
  "path": 
    {
        "root": ""
    },

  "network":
    {
        "norm_type": "batch",
#         "ngf": 64,
#         "ngb": 16,
#         "input_ngc": 1,
#         "output_ngc": 1
    },

  "train" : 
    {
        "manual_seed": 777,
        "niter": 20,
        "val_freq": 1,
        "lr": 1e-4,
        "criterion": "mse",
        "scheduler": dict(name="ReduceLROnPlateau", factor=0.2, threshold=0.1, patience=5, cooldown=2),
        "scheduler_frequency": 1
    },
    
  "logger": 
    {
        "print_freq": 100, # images
        "save_checkpoint_freq": 1 # iteration
    }
}


In [ ]:
def parse(opt_path, is_train=True):
#     with open(opt_path, 'r') as f:
#         opt = json.load(f, object_pairs_hook=OrderedDict)
    opt['is_train'] = is_train

    for key, path in opt['path'].items():
        opt['path'][key] = os.path.expanduser(path)
    if is_train:
        experiments_root = os.path.join(opt['path']['root'], 'experiments', opt['name'])
        opt['path']['experiments_root'] = experiments_root
        opt['path']['options'] = experiments_root
        opt['path']['trained_models'] = os.path.join(experiments_root, 'trained_models')
        opt['path']['log'] = os.path.join(experiments_root, 'log')
    else:
        results_root = os.path.join(opt['path']['root'], 'results', opt['name'])
        opt['path']['results_root'] = results_root
        opt['path']['log'] = os.path.join(results_root, 'log')
        opt['path']['test_images'] = os.path.join(results_root, 'test_images')

    return opt

opt = parse(opt)


In [ ]:
training_image_list = create_image_list(opt, train=True)
valid_image_list = create_image_list(opt, valid=True)


0/450 images loaded
50/450 images loaded
100/450 images loaded
150/450 images loaded
200/450 images loaded
250/450 images loaded
300/450 images loaded
350/450 images loaded
400/450 images loaded
0/34 images loaded


In [ ]:
training_set = create_dataset(opt["datasets"]["train"], training_image_list)
valid_set = create_dataset(opt["datasets"]["valid"], valid_image_list)


In [ ]:
from models.models import create_model
model = create_model(opt)

Model [ESPCNModel] is created.


In [ ]:
from utils.logger import Logger
logger = Logger(opt)

In [ ]:
def validate(val_set, val_size, model, logger, epoch, img_no):
    print('Start validation phase ...')
    val_start_time = time.time()
    model.eval() # Change to eval mode. It is important for BN layers.

    val_results = OrderedDict()
    avg_psnr = 0.0 
    avg_mse = 0.0
    for val_data in val_set:
      assert val_data['H'].shape[0] == val_data['L'].shape[0]
      batch_no = val_data['H'].shape[0]

      model.feed_data(val_data)
      model.forward()
      
      visuals = model.get_current_visuals()
      
      sr_img = visuals['super-resolution'] # uint8
      gt_img = visuals['ground-truth'] # uint8
      
      mse = model.criterion(sr_img, gt_img).item()
      psnr = 10 * log10(1 / mse)
      avg_mse += mse
      avg_psnr += psnr

    avg_mse = avg_mse / val_size
    avg_psnr = avg_psnr / val_size

    val_results['mse'] = avg_mse
    val_results['psnr'] = avg_psnr

    val_duration = time.time() - val_start_time

    model.train() # Change back to train mode.
    return val_results, val_duration


In [ ]:
# run this model
start_time = time.time()

val_size = len(valid_set)
depth = 624

avg_train_loss_list=[]
avg_valid_loss_list=[]
lr_list = []
psnr_list = []

for iteration in range(opt['train']['niter']+1):
  train_loss=0
  val_loss=0
  for i, train_data in enumerate(training_set):
    train_start_time = time.time()
    # training
    model.feed_data(train_data)
    model.optimize_parameters(i)
    train_duration = time.time() - train_start_time
    
    train_loss += model.loss.item() # this just gets a value, whereas get_current_losses gets a dict
    if i % opt['logger']['print_freq'] == 0:
      losses = model.get_current_losses()
      logger.print_results(losses, iteration, i, train_duration, 'loss')      
        
  avg_train_loss = train_loss/len(training_set)
  avg_train_loss_list.append(avg_train_loss)

  # write training results
  if i == len(training_set)-1:
    train_results = dict(avg_train_loss = avg_train_loss,
                         learning_rate = model.optimizer.state_dict()['param_groups'][0]['lr'])
    
    logger.print_results(train_results, iteration, i, 0, 'loss')

  # validation
  if iteration % opt['train']['val_freq'] == 0:
    val_results, val_duration = validate(valid_set, val_size, model, logger, iteration, i)
    print(val_results)
    # Save to log
    logger.print_results(val_results, iteration, i, val_duration, 'val')

  # save
  if iteration % opt['logger']['save_checkpoint_freq'] == 0:
    print('Saving the model at the end of iteration %d' % (iteration))
    model.save(iteration)

  # append to lists
  lr = model.optimizer.state_dict()['param_groups'][0]['lr']
  lr_list.append(lr)
  avg_valid_loss_list.append(val_results['mse'])
  psnr_list.append(val_results['psnr'])

  # update learning rate
  if opt['train'].get('scheduler') is not None:
    if iteration % opt['train'].get('scheduler_frequency') == 0:
      model.update_learning_rate(val_results['psnr'])

  print('end of iteration ' + str(iteration))

print('lr_list:', lr_list)
print('psnr_list:', psnr_list)
print('avg_train_loss_list', avg_train_loss_list)
print('avg_valid_loss_list', avg_valid_loss_list)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


(epoch:   0, iters:        0, time: 0.029) loss: 0.000301 
(epoch:   0, iters:      100, time: 0.029) loss: 0.000433 
(epoch:   0, iters:      200, time: 0.030) loss: 0.000387 
(epoch:   0, iters:      300, time: 0.029) loss: 0.000661 
(epoch:   0, iters:      400, time: 0.028) loss: 0.000700 
(epoch:   0, iters:      449, time: 0.000) avg_train_loss: 0.000464 learning_rate: 0.000100 
Start validation phase ...
avg_mse: 0.0004728010098915547
avg_psnr 33.35946924906156
OrderedDict([('mse', 0.0004728010098915547), ('psnr', 33.35946924906156)])
(epoch:   0, iters:      449, time: 51.703) mse: 0.000473 psnr: 33.359469 
Saving the model at the end of iteration 0
end of iteration 0
(epoch:   1, iters:        0, time: 0.034) loss: 0.000301 
(epoch:   1, iters:      100, time: 0.029) loss: 0.000319 
(epoch:   1, iters:      200, time: 0.028) loss: 0.000457 
(epoch:   1, iters:      300, time: 0.029) loss: 0.000665 
(epoch:   1, iters:      400, time: 0.028) loss: 0.000687 
(epoch:   1, iters: 

KeyboardInterrupt: ignored

In [ ]:
validate(valid_set, val_size, depth, model, logger, 1, 1)

Start validation phase ...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


(epoch:   1, iters:        1, time: 66.503) psnr: 34.756874 


In [ ]:
valid_set[0]['H'].shape

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([624, 1, 240, 240])

In [ ]:
avg_psnr = 0.0 
for val_data in valid_set:
    val_data_slice = dict(H=val_data['H'],
                          L=val_data['L'])
    model.feed_data(val_data_slice)
    model.forward()

    visuals = model.get_current_visuals()
    
    lr_img = visuals['low-resolution']
    sr_img = visuals['super-resolution'] # uint8
    gt_img = visuals['ground-truth'] # uint8

    print(lr_img.shape, sr_img.shape, gt_img.shape)

    mse = model.criterion(sr_img, gt_img).item()
    print(mse)
    psnr = 10 * log10(1 / mse)
    print(psnr)
    avg_psnr += psnr
avg_psnr = avg_psnr / val_size / depth
avg_psnr

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([624, 1, 60, 60]) torch.Size([624, 1, 240, 240]) torch.Size([624, 1, 240, 240])
0.0004941850202158093
33.06110423110817


KeyboardInterrupt: ignored

In [ ]:
model.optimizer.state_dict()['param_groups'][0]['lr']

0.0001

In [ ]:
a = model.get_current_losses()
a['lr'] = model.optimizer.state_dict()['param_groups'][0]['lr']
a

OrderedDict([('loss', 0.00041740122833289206), ('lr', 0.0001)])

In [ ]:
len(training_set)

450

In [ ]:
for i, data in enumerate(training_set):
  print(i)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


0
1
2
3


KeyboardInterrupt: ignored